# DuckDB Python 学习指南

## 目录
1. [DuckDB 简介](#1-duckdb-简介)
2. [安装与环境配置](#2-安装与环境配置)
3. [基础操作](#3-基础操作)
4. [数据操作（DML）](#4-数据操作dml)
5. [查询操作（DQL）](#5-查询操作dql)
6. [高级特性](#6-高级特性)
7. [与 MySQL 语法对比](#7-与-mysql-语法对比)
8. [实践练习](#8-实践练习)


## 1. DuckDB 简介

### 什么是 DuckDB？

DuckDB 是一个**内存中的分析型数据库管理系统（OLAP）**，专门为数据分析工作负载而设计。

### 主要特点：
- 🚀 **高性能**：列式存储，针对分析查询优化
- 💾 **零配置**：无需安装服务器，直接使用
- 📊 **SQL 兼容**：支持标准 SQL 语法
- 🔗 **易于集成**：可直接读取 CSV、Parquet、JSON 等格式
- 🐍 **Python 友好**：提供简洁的 Python API

### 适用场景：
- 数据分析与探索
- 数据科学项目
- 临时数据分析
- 数据转换与清洗
- 小到中等规模的数据集分析

### 与 MySQL 的主要区别：
- **MySQL**：关系型数据库，需要服务器，适合事务处理（OLTP）
- **DuckDB**：分析型数据库，无需服务器，适合数据分析（OLAP）


## 2. 安装与环境配置


In [1]:
# 安装 DuckDB
# 在命令行中运行: pip install duckdb

# 导入 DuckDB 库
import duckdb

# 检查版本
print(f"DuckDB 版本: {duckdb.__version__}")

# 注意：DuckDB 是嵌入式数据库，不需要像 MySQL 那样先启动服务器
# MySQL 需要：1. 安装 MySQL Server 2. 启动服务 3. 创建数据库 4. 连接
# DuckDB 只需要：1. pip install duckdb 2. 直接使用


DuckDB 版本: 1.4.0


## 3. 基础操作

### 3.1 连接数据库

DuckDB 支持两种模式：
- **内存数据库**：数据存储在内存中，程序结束后数据消失
- **持久化数据库**：数据保存在文件中，可以持久化存储


In [2]:
# 方式1：创建内存数据库（默认方式）
# 类似于 MySQL 的临时表，但这里是整个数据库都在内存中
conn = duckdb.connect()  # 或者 duckdb.connect(':memory:')
print("内存数据库连接成功！")

# 方式2：创建持久化数据库文件
# 类似于 MySQL 的数据库文件，但 DuckDB 是单个文件
# conn = duckdb.connect('my_database.duckdb')
# print("持久化数据库连接成功！")

# MySQL 对比：
# MySQL: conn = mysql.connector.connect(host='localhost', user='root', password='pass', database='dbname')
# DuckDB: conn = duckdb.connect('database.duckdb')  # 更简单，无需服务器配置


内存数据库连接成功！


In [3]:
# 获取游标对象（用于执行 SQL 语句）
cursor = conn.cursor()

# 执行简单的 SQL 查询
result = cursor.execute("SELECT 1 + 1 AS result").fetchone()
print(f"计算结果: {result[0]}")

# 或者使用更简洁的方式（推荐）
result = conn.execute("SELECT 'Hello DuckDB!' AS greeting").fetchone()
print(f"问候语: {result[0]}")


计算结果: 2
问候语: Hello DuckDB!


### 3.2 创建表（DDL - Data Definition Language）

DuckDB 支持标准的 SQL CREATE TABLE 语法。


In [4]:
# 创建学生表
# 语法与 MySQL 基本相同，但数据类型略有差异
conn.execute("""
    CREATE TABLE students (
        id INTEGER PRIMARY KEY,
        name VARCHAR(50) NOT NULL,
        age INTEGER,
        grade VARCHAR(10),
        score DECIMAL(5, 2)
    )
""")

print("学生表创建成功！")

# DuckDB 数据类型说明：
# - INTEGER: 整数（MySQL 中也有）
# - VARCHAR(n): 可变长度字符串（MySQL 中也有）
# - DECIMAL(p, s): 精确小数（MySQL 中也有）
# - DOUBLE: 双精度浮点数
# - DATE: 日期
# - TIMESTAMP: 时间戳
# - BOOLEAN: 布尔值

# MySQL 对比：
# MySQL 中通常需要指定存储引擎：CREATE TABLE ... ENGINE=InnoDB
# DuckDB 不需要指定，默认就是列式存储


学生表创建成功！


In [5]:
# 查看表结构
# MySQL 中使用: DESCRIBE table_name 或 SHOW COLUMNS FROM table_name
# DuckDB 使用: DESCRIBE 或 PRAGMA table_info()
result = conn.execute("DESCRIBE students").fetchall()
print("表结构：")
for row in result:
    print(row)


表结构：
('id', 'INTEGER', 'NO', 'PRI', None, None)
('name', 'VARCHAR', 'NO', None, None, None)
('age', 'INTEGER', 'YES', None, None, None)
('grade', 'VARCHAR', 'YES', None, None, None)
('score', 'DECIMAL(5,2)', 'YES', None, None, None)


## 4. 数据操作（DML - Data Manipulation Language）

### 4.1 插入数据（INSERT）


In [6]:
# 方式1：单条插入（与 MySQL 语法相同）
conn.execute("""
    INSERT INTO students (id, name, age, grade, score)
    VALUES (1, '张三', 18, '一年级', 95.5)
""")

# 方式2：批量插入（推荐，性能更好）
students_data = [
    (2, '李四', 19, '二年级', 88.0),
    (3, '王五', 20, '三年级', 92.5),
    (4, '赵六', 18, '一年级', 87.5),
    (5, '钱七', 21, '四年级', 96.0),
]

conn.executemany("""
    INSERT INTO students (id, name, age, grade, score)
    VALUES (?, ?, ?, ?, ?)
""", students_data)

print("数据插入成功！")

# MySQL 对比：
# MySQL 批量插入: INSERT INTO ... VALUES (...), (...), (...)
# DuckDB 也支持这种方式，但使用 executemany 更高效


数据插入成功！


In [ ]:
# 方式3：从 Pandas DataFrame 插入（DuckDB 的特色功能）
import pandas as pd

# 创建 DataFrame
df_new = pd.DataFrame({
    'id': [6, 7, 8],
    'name': ['孙八', '周九', '吴十'],
    'age': [19, 20, 18],
    'grade': ['二年级', '三年级', '一年级'],
    'score': [89.5, 91.0, 94.5]
})

# 直接插入 DataFrame（非常方便！）
conn.execute("INSERT INTO students SELECT * FROM df_new")

print("从 DataFrame 插入数据成功！")

# MySQL 对比：
# MySQL 需要先将 DataFrame 转换为 SQL 语句或使用其他库
# DuckDB 可以直接操作 DataFrame，这是它的优势之一


### 4.2 查询数据（SELECT）

查询语法与 MySQL 基本相同。


In [ ]:
# 查询所有数据
result = conn.execute("SELECT * FROM students").fetchall()
print("所有学生数据：")
for row in result:
    print(row)


In [ ]:
# 将查询结果转换为 Pandas DataFrame（DuckDB 的特色功能）
df = conn.execute("SELECT * FROM students").df()
print("\n转换为 DataFrame：")
print(df)
print(f"\n数据类型：\n{df.dtypes}")


In [ ]:
# 条件查询
result = conn.execute("""
    SELECT name, age, score 
    FROM students 
    WHERE score > 90
    ORDER BY score DESC
""").fetchall()

print("分数大于 90 的学生：")
for row in result:
    print(f"姓名: {row[0]}, 年龄: {row[1]}, 分数: {row[2]}")


In [ ]:
# 聚合查询
result = conn.execute("""
    SELECT 
        grade,
        COUNT(*) AS student_count,
        AVG(score) AS avg_score,
        MAX(score) AS max_score,
        MIN(score) AS min_score
    FROM students
    GROUP BY grade
    ORDER BY avg_score DESC
""").df()

print("各年级统计信息：")
print(result)

# MySQL 对比：语法完全相同


### 4.3 更新数据（UPDATE）


In [ ]:
# 更新数据（语法与 MySQL 相同）
conn.execute("""
    UPDATE students 
    SET score = score + 5 
    WHERE grade = '一年级'
""")

print("更新完成！")

# 查看更新后的数据
df_updated = conn.execute("SELECT * FROM students WHERE grade = '一年级'").df()
print("\n更新后的一年级学生：")
print(df_updated)

# MySQL 对比：语法完全相同


### 4.4 删除数据（DELETE）


In [ ]:
# 删除数据（语法与 MySQL 相同）
# 注意：先查看要删除的数据
print("删除前的数据：")
df_before = conn.execute("SELECT * FROM students WHERE score < 85").df()
print(df_before)

# 执行删除
conn.execute("DELETE FROM students WHERE score < 85")

print("\n删除完成！")

# 验证删除结果
df_after = conn.execute("SELECT * FROM students").df()
print(f"\n剩余学生数量: {len(df_after)}")

# MySQL 对比：语法完全相同
# 注意：MySQL 中可能需要 COMMIT（如果使用事务），DuckDB 默认自动提交


## 5. 查询操作（DQL - Data Query Language）

### 5.1 连接查询（JOIN）


In [ ]:
# 创建课程表
conn.execute("""
    CREATE TABLE courses (
        course_id INTEGER PRIMARY KEY,
        course_name VARCHAR(50),
        teacher VARCHAR(50)
    )
""")

# 创建选课表
conn.execute("""
    CREATE TABLE enrollments (
        student_id INTEGER,
        course_id INTEGER,
        enrollment_date DATE,
        PRIMARY KEY (student_id, course_id)
    )
""")

# 插入示例数据
conn.execute("""
    INSERT INTO courses VALUES
    (1, '数学', '张老师'),
    (2, '英语', '李老师'),
    (3, '物理', '王老师')
""")

conn.execute("""
    INSERT INTO enrollments VALUES
    (1, 1, '2024-01-15'),
    (1, 2, '2024-01-16'),
    (2, 1, '2024-01-15'),
    (2, 3, '2024-01-17'),
    (3, 2, '2024-01-16')
""")

print("表创建和数据插入完成！")


In [ ]:
# INNER JOIN（内连接）
result = conn.execute("""
    SELECT 
        s.name AS student_name,
        c.course_name,
        c.teacher,
        e.enrollment_date
    FROM students s
    INNER JOIN enrollments e ON s.id = e.student_id
    INNER JOIN courses c ON e.course_id = c.course_id
    ORDER BY s.name, c.course_name
""").df()

print("学生选课信息（内连接）：")
print(result)

# MySQL 对比：语法完全相同


In [ ]:
# LEFT JOIN（左连接）
result = conn.execute("""
    SELECT 
        s.name AS student_name,
        COALESCE(c.course_name, '未选课') AS course_name
    FROM students s
    LEFT JOIN enrollments e ON s.id = e.student_id
    LEFT JOIN courses c ON e.course_id = c.course_id
    ORDER BY s.name
""").df()

print("所有学生的选课情况（左连接）：")
print(result)

# MySQL 对比：语法完全相同
# COALESCE 函数在 MySQL 和 DuckDB 中都可用


### 5.2 子查询（Subquery）


In [ ]:
# 使用子查询查找分数高于平均分的学生
result = conn.execute("""
    SELECT name, score
    FROM students
    WHERE score > (SELECT AVG(score) FROM students)
    ORDER BY score DESC
""").df()

print("分数高于平均分的学生：")
print(result)

# MySQL 对比：语法完全相同


In [ ]:
# 使用 EXISTS 子查询
result = conn.execute("""
    SELECT s.name, s.score
    FROM students s
    WHERE EXISTS (
        SELECT 1 
        FROM enrollments e 
        WHERE e.student_id = s.id
    )
    ORDER BY s.score DESC
""").df()

print("已选课的学生：")
print(result)

# MySQL 对比：语法完全相同


### 5.3 窗口函数（Window Functions）

DuckDB 支持强大的窗口函数，与 MySQL 8.0+ 类似。


In [ ]:
# ROW_NUMBER() - 为每行分配序号
result = conn.execute("""
    SELECT 
        name,
        score,
        grade,
        ROW_NUMBER() OVER (PARTITION BY grade ORDER BY score DESC) AS rank_in_grade
    FROM students
    ORDER BY grade, score DESC
""").df()

print("各年级学生排名：")
print(result)


In [ ]:
# RANK() 和 DENSE_RANK()
result = conn.execute("""
    SELECT 
        name,
        score,
        RANK() OVER (ORDER BY score DESC) AS rank_with_gaps,
        DENSE_RANK() OVER (ORDER BY score DESC) AS rank_no_gaps
    FROM students
    ORDER BY score DESC
""").df()

print("学生排名对比：")
print(result)

# MySQL 对比：
# MySQL 8.0+ 支持窗口函数，语法相同
# MySQL 5.7 及以下版本不支持窗口函数


## 6. 高级特性

### 6.1 直接读取 CSV 文件

这是 DuckDB 的一大特色，可以直接查询 CSV 文件，无需先导入数据库。


In [ ]:
# 首先创建一个示例 CSV 文件
import csv

csv_data = [
    ['product_id', 'product_name', 'price', 'category'],
    [1, '笔记本电脑', 5999.00, '电子产品'],
    [2, '鼠标', 99.00, '电子产品'],
    [3, '键盘', 299.00, '电子产品'],
    [4, '办公桌', 899.00, '家具'],
    [5, '椅子', 599.00, '家具'],
]

with open('products.csv', 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(csv_data)

print("CSV 文件创建成功！")


In [ ]:
# 直接查询 CSV 文件（无需先导入！）
result = conn.execute("""
    SELECT 
        category,
        COUNT(*) AS product_count,
        AVG(price) AS avg_price
    FROM read_csv_auto('products.csv')
    GROUP BY category
""").df()

print("从 CSV 文件直接查询结果：")
print(result)

# MySQL 对比：
# MySQL 需要：1. 创建表 2. 使用 LOAD DATA INFILE 或导入工具 3. 然后查询
# DuckDB 可以直接查询文件，非常方便！


In [ ]:
# 将 CSV 数据导入到表中
conn.execute("""
    CREATE TABLE products AS
    SELECT * FROM read_csv_auto('products.csv')
""")

# 查看导入的数据
df_products = conn.execute("SELECT * FROM products").df()
print("导入的产品数据：")
print(df_products)


### 6.2 与 Pandas 深度集成


In [ ]:
# 创建示例 DataFrame
sales_df = pd.DataFrame({
    'date': pd.date_range('2024-01-01', periods=10, freq='D'),
    'product_id': [1, 2, 1, 3, 2, 1, 3, 2, 1, 3],
    'quantity': [10, 5, 8, 12, 6, 9, 7, 4, 11, 8],
    'revenue': [1000, 500, 800, 1200, 600, 900, 700, 400, 1100, 800]
})

print("原始 DataFrame：")
print(sales_df.head())


In [ ]:
# 直接在 SQL 中使用 DataFrame（无需先导入数据库！）
result = conn.execute("""
    SELECT 
        product_id,
        SUM(quantity) AS total_quantity,
        SUM(revenue) AS total_revenue,
        AVG(revenue) AS avg_revenue
    FROM sales_df
    GROUP BY product_id
    ORDER BY total_revenue DESC
""").df()

print("使用 SQL 查询 DataFrame 的结果：")
print(result)

# MySQL 对比：
# MySQL 无法直接查询 DataFrame，需要先导入
# DuckDB 可以直接在 SQL 中引用 DataFrame，非常强大！


### 6.3 读取 Parquet 文件

Parquet 是一种高效的列式存储格式，DuckDB 原生支持。


In [ ]:
# 将 DataFrame 保存为 Parquet 格式
sales_df.to_parquet('sales.parquet', index=False)
print("Parquet 文件创建成功！")

# 直接从 Parquet 文件查询
result = conn.execute("""
    SELECT 
        DATE_TRUNC('week', date) AS week,
        SUM(revenue) AS weekly_revenue
    FROM read_parquet('sales.parquet')
    GROUP BY week
    ORDER BY week
""").df()

print("从 Parquet 文件查询的周收入：")
print(result)

# MySQL 对比：
# MySQL 不支持直接读取 Parquet 文件
# 需要先转换为 CSV 或其他格式再导入


### 6.4 事务处理

DuckDB 支持事务，但默认是自动提交模式。


In [ ]:
# 开始事务
conn.begin()

try:
    # 执行多个操作
    conn.execute("INSERT INTO students VALUES (9, '测试学生', 20, '测试年级', 85.0)")
    conn.execute("UPDATE students SET score = 100 WHERE id = 9")
    
    # 提交事务
    conn.commit()
    print("事务提交成功！")
    
except Exception as e:
    # 回滚事务
    conn.rollback()
    print(f"事务回滚：{e}")

# MySQL 对比：
# MySQL: conn.autocommit = False 然后 conn.commit() 或 conn.rollback()
# DuckDB: conn.begin() 然后 conn.commit() 或 conn.rollback()
# 语法略有不同，但概念相同


## 7. 与 MySQL 语法对比

### 主要差异总结


| 特性 | MySQL | DuckDB |
|------|-------|--------|
| **安装** | 需要安装服务器 | pip install duckdb |
| **连接** | 需要服务器地址、用户名、密码 | 直接连接文件或内存 |
| **数据库文件** | 多个文件/目录 | 单个 .duckdb 文件 |
| **数据类型** | 更多类型（如 TINYINT, MEDIUMINT） | 标准类型（INTEGER, VARCHAR等） |
| **AUTO_INCREMENT** | 支持 | 不支持（使用序列） |
| **存储引擎** | 需要指定（InnoDB, MyISAM等） | 默认列式存储 |
| **直接读文件** | 不支持 | 支持 CSV, Parquet, JSON |
| **Pandas 集成** | 需要额外库 | 原生支持 |
| **窗口函数** | MySQL 8.0+ 支持 | 完全支持 |
| **事务** | 支持（默认自动提交） | 支持（默认自动提交） |
| **索引** | 支持多种索引 | 自动优化，无需手动创建 |
| **主键自增** | AUTO_INCREMENT | SERIAL 或手动管理 |


In [ ]:
# 示例：MySQL vs DuckDB 创建自增主键

# MySQL 方式：
# CREATE TABLE users (
#     id INT AUTO_INCREMENT PRIMARY KEY,
#     name VARCHAR(50)
# );

# DuckDB 方式1：使用 SERIAL（推荐）
conn.execute("""
    CREATE TABLE users_duckdb (
        id INTEGER PRIMARY KEY,
        name VARCHAR(50)
    )
""")

# DuckDB 方式2：手动管理 ID（更灵活）
# 可以创建一个序列或使用 MAX(id) + 1

print("注意：DuckDB 没有 AUTO_INCREMENT，需要手动管理 ID 或使用序列")


## 8. 实践练习

### 练习 1：学生成绩管理系统

创建一个完整的学生成绩管理系统，包括：
1. 创建学生表和成绩表
2. 插入测试数据
3. 查询各科平均分
4. 找出成绩最好的学生
5. 统计不及格学生


In [ ]:
# 练习 1 解答

# 1. 创建表
conn.execute("""
    CREATE TABLE student_scores (
        student_id INTEGER,
        student_name VARCHAR(50),
        subject VARCHAR(50),
        score DECIMAL(5, 2),
        exam_date DATE
    )
""")

# 2. 插入测试数据
scores_data = [
    (1, '张三', '数学', 95.5, '2024-01-15'),
    (1, '张三', '英语', 88.0, '2024-01-16'),
    (1, '张三', '物理', 92.0, '2024-01-17'),
    (2, '李四', '数学', 78.5, '2024-01-15'),
    (2, '李四', '英语', 85.0, '2024-01-16'),
    (2, '李四', '物理', 82.5, '2024-01-17'),
    (3, '王五', '数学', 96.0, '2024-01-15'),
    (3, '王五', '英语', 90.5, '2024-01-16'),
    (3, '王五', '物理', 94.0, '2024-01-17'),
    (4, '赵六', '数学', 65.0, '2024-01-15'),
    (4, '赵六', '英语', 72.0, '2024-01-16'),
    (4, '赵六', '物理', 68.5, '2024-01-17'),
]

conn.executemany("""
    INSERT INTO student_scores (student_id, student_name, subject, score, exam_date)
    VALUES (?, ?, ?, ?, ?)
""", scores_data)

print("数据插入完成！")


In [ ]:
# 3. 查询各科平均分
avg_scores = conn.execute("""
    SELECT 
        subject,
        COUNT(*) AS student_count,
        AVG(score) AS avg_score,
        MAX(score) AS max_score,
        MIN(score) AS min_score
    FROM student_scores
    GROUP BY subject
    ORDER BY avg_score DESC
""").df()

print("各科成绩统计：")
print(avg_scores)


In [ ]:
# 4. 找出成绩最好的学生（平均分最高）
top_student = conn.execute("""
    SELECT 
        student_name,
        AVG(score) AS avg_score,
        COUNT(*) AS subject_count
    FROM student_scores
    GROUP BY student_name
    ORDER BY avg_score DESC
    LIMIT 1
""").df()

print("成绩最好的学生：")
print(top_student)


In [ ]:
# 5. 统计不及格学生（分数 < 60）
failing_students = conn.execute("""
    SELECT 
        student_name,
        subject,
        score
    FROM student_scores
    WHERE score < 60
    ORDER BY student_name, subject
""").df()

print("不及格学生：")
if len(failing_students) > 0:
    print(failing_students)
else:
    print("没有不及格的学生！")


### 练习 2：数据分析任务

使用 DuckDB 分析销售数据：
1. 从 CSV 文件读取数据
2. 计算每日销售额
3. 找出销售额最高的产品
4. 计算月度增长率


In [ ]:
# 练习 2 解答

# 1. 创建销售数据 CSV
sales_data = [
    ['sale_id', 'date', 'product', 'quantity', 'unit_price'],
    [1, '2024-01-01', '产品A', 10, 100.0],
    [2, '2024-01-01', '产品B', 5, 200.0],
    [3, '2024-01-02', '产品A', 8, 100.0],
    [4, '2024-01-02', '产品C', 12, 150.0],
    [5, '2024-01-03', '产品B', 15, 200.0],
    [6, '2024-01-03', '产品A', 20, 100.0],
    [7, '2024-02-01', '产品A', 15, 100.0],
    [8, '2024-02-01', '产品B', 10, 200.0],
    [9, '2024-02-02', '产品C', 8, 150.0],
    [10, '2024-02-03', '产品A', 25, 100.0],
]

with open('sales_data.csv', 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(sales_data)

print("销售数据 CSV 创建成功！")


In [ ]:
# 2. 计算每日销售额
daily_sales = conn.execute("""
    SELECT 
        date,
        SUM(quantity * unit_price) AS daily_revenue,
        SUM(quantity) AS total_quantity
    FROM read_csv_auto('sales_data.csv')
    GROUP BY date
    ORDER BY date
""").df()

print("每日销售额：")
print(daily_sales)


In [ ]:
# 3. 找出销售额最高的产品
top_product = conn.execute("""
    SELECT 
        product,
        SUM(quantity * unit_price) AS total_revenue,
        SUM(quantity) AS total_quantity
    FROM read_csv_auto('sales_data.csv')
    GROUP BY product
    ORDER BY total_revenue DESC
    LIMIT 1
""").df()

print("销售额最高的产品：")
print(top_product)


In [ ]:
# 4. 计算月度增长率
monthly_sales = conn.execute("""
    SELECT 
        DATE_TRUNC('month', CAST(date AS DATE)) AS month,
        SUM(quantity * unit_price) AS monthly_revenue
    FROM read_csv_auto('sales_data.csv')
    GROUP BY month
    ORDER BY month
""").df()

# 计算增长率
monthly_sales['growth_rate'] = monthly_sales['monthly_revenue'].pct_change() * 100

print("月度销售额及增长率：")
print(monthly_sales)

# 注意：这里使用了 Pandas 的 pct_change() 方法
# 也可以使用 SQL 窗口函数来计算增长率


In [ ]:
# 使用 SQL 窗口函数计算月度增长率（更优雅的方式）
monthly_growth = conn.execute("""
    WITH monthly_revenue AS (
        SELECT 
            DATE_TRUNC('month', CAST(date AS DATE)) AS month,
            SUM(quantity * unit_price) AS monthly_revenue
        FROM read_csv_auto('sales_data.csv')
        GROUP BY month
    )
    SELECT 
        month,
        monthly_revenue,
        LAG(monthly_revenue) OVER (ORDER BY month) AS prev_month_revenue,
        ROUND(
            (monthly_revenue - LAG(monthly_revenue) OVER (ORDER BY month)) * 100.0 
            / LAG(monthly_revenue) OVER (ORDER BY month), 
            2
        ) AS growth_rate_percent
    FROM monthly_revenue
    ORDER BY month
""").df()

print("使用 SQL 计算的月度增长率：")
print(monthly_growth)


### 练习 3：综合应用

创建一个员工管理系统，实现：
1. 员工信息表（包含部门）
2. 薪资表
3. 查询各部门平均薪资
4. 找出薪资最高的员工
5. 计算薪资排名


In [ ]:
# 练习 3 解答

# 1. 创建员工表
conn.execute("""
    CREATE TABLE employees (
        emp_id INTEGER PRIMARY KEY,
        emp_name VARCHAR(50),
        department VARCHAR(50),
        hire_date DATE
    )
""")

# 2. 创建薪资表
conn.execute("""
    CREATE TABLE salaries (
        emp_id INTEGER,
        salary DECIMAL(10, 2),
        bonus DECIMAL(10, 2),
        salary_date DATE,
        FOREIGN KEY (emp_id) REFERENCES employees(emp_id)
    )
""")

# 插入员工数据
employees_data = [
    (1, '张经理', '技术部', '2020-01-15'),
    (2, '李工程师', '技术部', '2021-03-20'),
    (3, '王主管', '销售部', '2019-06-10'),
    (4, '赵销售', '销售部', '2022-01-05'),
    (5, '钱会计', '财务部', '2020-09-12'),
    (6, '孙出纳', '财务部', '2021-11-18'),
]

conn.executemany("""
    INSERT INTO employees (emp_id, emp_name, department, hire_date)
    VALUES (?, ?, ?, ?)
""", employees_data)

# 插入薪资数据
salaries_data = [
    (1, 15000.00, 5000.00, '2024-01-01'),
    (2, 12000.00, 3000.00, '2024-01-01'),
    (3, 13000.00, 4000.00, '2024-01-01'),
    (4, 10000.00, 2000.00, '2024-01-01'),
    (5, 11000.00, 2500.00, '2024-01-01'),
    (6, 9000.00, 1500.00, '2024-01-01'),
]

conn.executemany("""
    INSERT INTO salaries (emp_id, salary, bonus, salary_date)
    VALUES (?, ?, ?, ?)
""", salaries_data)

print("员工和薪资数据插入完成！")


In [ ]:
# 3. 查询各部门平均薪资
dept_avg_salary = conn.execute("""
    SELECT 
        e.department,
        COUNT(DISTINCT e.emp_id) AS employee_count,
        AVG(s.salary + s.bonus) AS avg_total_income,
        SUM(s.salary + s.bonus) AS total_payroll
    FROM employees e
    INNER JOIN salaries s ON e.emp_id = s.emp_id
    GROUP BY e.department
    ORDER BY avg_total_income DESC
""").df()

print("各部门薪资统计：")
print(dept_avg_salary)


In [ ]:
# 4. 找出薪资最高的员工
top_employee = conn.execute("""
    SELECT 
        e.emp_name,
        e.department,
        s.salary,
        s.bonus,
        (s.salary + s.bonus) AS total_income
    FROM employees e
    INNER JOIN salaries s ON e.emp_id = s.emp_id
    ORDER BY total_income DESC
    LIMIT 1
""").df()

print("薪资最高的员工：")
print(top_employee)


In [ ]:
# 5. 计算薪资排名（使用窗口函数）
salary_ranking = conn.execute("""
    SELECT 
        e.emp_name,
        e.department,
        s.salary + s.bonus AS total_income,
        RANK() OVER (ORDER BY s.salary + s.bonus DESC) AS overall_rank,
        RANK() OVER (PARTITION BY e.department ORDER BY s.salary + s.bonus DESC) AS dept_rank
    FROM employees e
    INNER JOIN salaries s ON e.emp_id = s.emp_id
    ORDER BY total_income DESC
""").df()

print("员工薪资排名：")
print(salary_ranking)


## 总结

### DuckDB 的优势

1. **简单易用**：无需配置服务器，直接使用
2. **高性能**：列式存储，分析查询速度快
3. **文件集成**：可直接读取 CSV、Parquet、JSON 等格式
4. **Pandas 友好**：与 Python 数据分析生态完美集成
5. **SQL 兼容**：支持标准 SQL，学习成本低

### 适用场景

✅ **适合使用 DuckDB：**
- 数据分析与探索
- 数据科学项目
- 临时数据分析任务
- 中小规模数据集分析
- 需要快速原型开发

❌ **不适合使用 DuckDB：**
- 需要高并发事务处理
- 需要复杂的用户权限管理
- 需要与其他系统深度集成
- 超大规模数据（TB+级别）

### 学习建议

1. 掌握基本 SQL 语法（与 MySQL 类似）
2. 熟悉 Pandas 集成功能
3. 学习直接读取文件的方法
4. 了解窗口函数等高级特性
5. 多做实践练习，熟悉各种场景

### 下一步学习

- 深入学习窗口函数
- 学习性能优化技巧
- 了解 DuckDB 的扩展功能
- 探索与其他 Python 库的集成（如 Polars、Dask）

---

**祝你学习愉快！** 🎉


In [ ]:
# 清理资源（可选）
# 如果使用文件数据库，记得关闭连接
# conn.close()

# 清理创建的 CSV 文件（可选）
import os
# os.remove('products.csv')
# os.remove('sales_data.csv')

print("学习完成！记得在实际项目中根据需求选择合适的数据库。")
